In [1]:
# 패키지 설치
!pip install finance-datareader

In [10]:
# date 변환
import pandas as pd
import FinanceDataReader as fdr
from datetime import timedelta
import os

ticker = '005930'   # 삼성전자 티커가 005930!
start_year = 2015
end_year = 2024
start_date = f'{start_year}-01-01'
end_date = '2024-08-29'
stock_data = fdr.DataReader(ticker, start_date, end_date)
valid_dates = stock_data.index.date

# 기사 날짜 조정 함수
def adjust_news_dates(row):
    try:
        # 문자열 형식을 datetime으로 변환
        article_datetime = pd.to_datetime(row['date'])
        
        # 16시 이상 또는 15시 20분 이상인 경우 다음 날로 조정
        if (article_datetime.hour > 15) or (article_datetime.hour == 15 and article_datetime.minute >= 20):
            article_datetime += timedelta(days=1)
        
        # 주식 데이터 인덱스에 있는 날짜 찾기
        while article_datetime.date() not in valid_dates:
            article_datetime += timedelta(days=1)
            # fdr의 stock_data.index.date에 없으면 다음날로 배정
        
        return article_datetime.date()
    
    except Exception as e:  # 예외처리
        print(f"Error processing date '{row['date']}': {e}")
        return None   # 잘못된 날짜는 None으로 반환

news_df = pd.read_csv('final_매일경제.csv')

# 기사 날짜 조정
news_df['adjusted_date'] = news_df.apply(adjust_news_dates, axis=1)

# csv 파일로 내보내기
output_filename = 'date_scaled_매일경제.csv'
news_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("날짜 변환 완료")

Error processing date '2024-08-28 15:56': Out of bounds nanosecond timestamp: 9223430160000000000
Error processing date '2024-08-28 16:23': Out of bounds nanosecond timestamp: 9223431780000000000
Error processing date '2024-08-28 16:46': Out of bounds nanosecond timestamp: 9223433160000000000
Error processing date '2024-08-28 16:47': Out of bounds nanosecond timestamp: 9223433220000000000
Error processing date '2024-08-28 17:58': Out of bounds nanosecond timestamp: 9223437480000000000
Error processing date '2024-08-28 18:03': Out of bounds nanosecond timestamp: 9223437780000000000
Error processing date '2024-08-28 18:05': Out of bounds nanosecond timestamp: 9223437900000000000
날짜 변환 완료


In [11]:
count_df = news_df.groupby(['adjusted_date', 'label']).size().unstack(fill_value=0)
ratio_df = count_df.div(count_df.sum(axis=1), axis=0); ratio_df

label,0,1,2
adjusted_date,,,
2015-01-02,0.312500,0.437500,0.250000
2015-01-05,0.444444,0.222222,0.333333
2015-01-06,0.214286,0.571429,0.214286
2015-01-07,0.307692,0.153846,0.538462
2015-01-08,0.400000,0.533333,0.066667
...,...,...,...
2024-08-22,0.250000,0.750000,0.000000
2024-08-23,0.000000,0.833333,0.166667
2024-08-26,0.000000,0.833333,0.166667


In [13]:
ratio_df.columns = ['neutral','positive','negative']

In [15]:
ratio_df.to_csv('new_sentiment_data.csv', encoding='utf-8-sig')

# 굿